# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [58]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [61]:

vacation_df=pd.read_csv("../WeatherPy/Output/weather_data.csv")
vacation_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,WindSpeed,Country,Date
0,0,Ključ,44.5350,16.7722,53.55,94,100,3.02,BA,1634931893
1,1,Bilibino,68.0546,166.4372,-4.31,100,100,1.30,RU,1634931893
2,2,Katsuura,35.1333,140.3000,55.51,81,31,14.52,JP,1634931894
3,3,Road Town,18.4167,-64.6167,87.89,62,20,10.36,VG,1634931894
4,4,Cockburn Town,21.4612,-71.1419,82.36,78,40,6.91,TC,1634931894
...,...,...,...,...,...,...,...,...,...,...
558,558,Dunedin,-45.8742,170.5036,53.80,69,100,11.99,NZ,1634932110
559,559,Taoudenni,22.6783,-3.9836,82.65,21,0,4.43,ML,1634932111
560,560,Petropavlovskoye,52.0712,84.1070,39.34,53,24,12.88,RU,1634932111
561,561,Novovasylivka,46.8315,35.7536,55.00,72,95,7.94,UA,1634932112


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [63]:
# Use the Lat and Lng as locations and Humidity as the weight.
fig = gmaps.figure()
locations = vacation_df[["Lat", "Lng"]]

humid_rate = vacation_df["Humidity"]


# Add Heatmap layer to map


heat_layer = gmaps.heatmap_layer(locations, weights=humid_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [64]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be 
# sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit 
#     the number of rows returned by your API requests to a reasonable number.

# new_vacation_df=vacation_df.copy()
#ideal condition
# vacation_df.loc[((vacation_df["Max Temp"] < 80) &  (vacation_df["Max Temp"] > 70)) 
#                 & (vacation_df["WindSpeed"] < 10) & (vacation_df["Cloudiness"] = 0)]


new_vacation_df=vacation_df.loc[((vacation_df["Max Temp"] <= 80) &  (vacation_df["Max Temp"] >= 70)) & (vacation_df["WindSpeed"] <= 10) & (vacation_df["Cloudiness"] == 0)].dropna()

new_vacation_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,WindSpeed,Country,Date
23,23,Cachoeira do Sul,-30.0392,-52.8939,77.07,52,0,4.68,BR,1634931901
56,56,Dzaoudzi,-12.7887,45.2699,75.09,83,0,3.44,YT,1634931917
117,117,Saint-Pierre,-21.3393,55.4781,73.29,83,0,4.61,RE,1634931940
137,137,Alice Springs,-23.7000,133.8833,74.75,23,0,9.22,AU,1634931749
265,265,Maputo,-25.9653,32.5892,73.22,85,0,5.99,MZ,1634931998
318,318,Quelimane,-17.8786,36.8883,75.15,69,0,8.01,MZ,1634932019
404,404,Artigas,-30.4000,-56.4667,77.56,45,0,9.04,UY,1634931986
407,407,Salalah,17.0151,54.0924,77.09,73,0,4.61,OM,1634931914
497,497,Mīnāb,27.1467,57.0801,78.30,33,0,6.51,IR,1634932087
549,549,Riyadh,24.6877,46.7219,77.14,29,0,6.98,SA,1634932106


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [65]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
hotel_df=new_vacation_df.loc[:,["City","Country","Lat","Lng"]]

hotel_df["Hotel Name"] =""

hotel_df

,City,Country,Lat,Lng,Hotel Name
23,Cachoeira do Sul,BR,-30.0392,-52.8939,
56,Dzaoudzi,YT,-12.7887,45.2699,
117,Saint-Pierre,RE,-21.3393,55.4781,
137,Alice Springs,AU,-23.7000,133.8833,
265,Maputo,MZ,-25.9653,32.5892,
318,Quelimane,MZ,-17.8786,36.8883,
404,Artigas,UY,-30.4000,-56.4667,
407,Salalah,OM,17.0151,54.0924,
497,Mīnāb,IR,27.1467,57.0801,
549,Riyadh,SA,24.6877,46.7219,


In [66]:
# Set parameters to search for hotels with 5000 meters.
params={
        
        "radius": 5000,
        "type":"hotel",
        "key":g_key
    }

In [67]:
#hotelslist=[]
#for i in range(len(hotel_df)):
for index,row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    city_name = row['City']
    
    params["location"]=f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    resp=requests.get(base_url,params=params)
    resp_json=resp.json()
    try:
        hotel_df["Hotel Name"]=resp_json["results"][0]['name']
        #hotelslist.append(resp_json["results"][0]['name'])
    except:
        
        print("Could not find result")


        

In [68]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
23,Cachoeira do Sul,BR,-30.0392,-52.8939,Riyadh
56,Dzaoudzi,YT,-12.7887,45.2699,Riyadh
117,Saint-Pierre,RE,-21.3393,55.4781,Riyadh
137,Alice Springs,AU,-23.7000,133.8833,Riyadh
265,Maputo,MZ,-25.9653,32.5892,Riyadh
318,Quelimane,MZ,-17.8786,36.8883,Riyadh
404,Artigas,UY,-30.4000,-56.4667,Riyadh
407,Salalah,OM,17.0151,54.0924,Riyadh
497,Mīnāb,IR,27.1467,57.0801,Riyadh
549,Riyadh,SA,24.6877,46.7219,Riyadh


In [69]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_information = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]





In [70]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations,info_box_content=hotel_information)
fig.add_layer(markers)
fig




Figure(layout=FigureLayout(height='420px'))